In [1]:
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [2]:
CLIENT_ID = 'GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR' # your Foursquare ID
CLIENT_SECRET = 'MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR
CLIENT_SECRET:MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3


In [3]:
#source from: https://en.wikipedia.org/wiki/Boroughs_of_New_York_City

from collections import OrderedDict
stac = {'Borough':["Bronx", "Brooklyn", "Manhattan", "Queens", "Staten Island"],
        'Population':[1471160, 2648771, 1664727, 2358582, 479458],
        'GDP':[28.787, 63.303, 629.682, 73.842, 11.249],
        'per capita':[19570, 23900, 378250, 31310, 23460],
        'square miles':[42.1, 70.82, 22.83, 108.53, 58.37],
        'persons /sq.mi':[34653, 37137, 72033, 21460, 8112]}
df_ny = pd.DataFrame.from_dict(stac)

In [4]:
df_ny

,Borough,Population,GDP,per capita,square miles,persons /sq.mi
0,Bronx,1471160,28.787,19570,42.10,34653
1,Brooklyn,2648771,63.303,23900,70.82,37137
2,Manhattan,1664727,629.682,378250,22.83,72033
3,Queens,2358582,73.842,31310,108.53,21460
4,Staten Island,479458,11.249,23460,58.37,8112


In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [8]:
neighborhoods_data = newyork_data['features']

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [12]:
df_nygeo = pd.merge(neighborhoods, df_ny, on='Borough', how = 'left')
df_nygeo.shape

(306, 9)

In [13]:
brooklyn_df = df_nygeo.loc[df_nygeo.Borough=="Brooklyn"].reset_index(drop=True)
brooklyn_df.shape

(70, 9)

In [14]:
address_1 = 'Brooklyn, NY'

geolocator_1 = Nominatim(user_agent="br_explorer")
location_1 = geolocator_1.geocode(address_1)
latitude_1 = location_1.latitude
longitude_1 = location_1.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude_1, longitude_1))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [15]:
search_query = 'restaurant' 
radius = 5000
print(search_query + ' .... OK!')

restaurant .... OK!


In [16]:
url_1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_1, longitude_1, VERSION, search_query, radius, LIMIT)
url_1

'https://api.foursquare.com/v2/venues/search?client_id=GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR&client_secret=MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3&ll=40.6501038,-73.9495823&v=20180604&query=restaurant&radius=5000&limit=300'

In [17]:
results_1 = requests.get(url_1).json()

In [18]:
# assign relevant part of JSON to venues
venues_1 = results_1['response']['venues']

# tranform venues into a dataframe
dataframe_1 = json_normalize(venues_1)
dataframe_1.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood
0,50b91d3de4b0802eb8488f39,Kreyol Flavor Bakery & Restaurant,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",v-1564173857,False,2816 Church Ave,40.650820,-73.950777,"[{'label': 'display', 'lat': 40.65082009251775...",128,11226,US,Brooklyn,NY,United States,"[2816 Church Ave, Brooklyn, NY 11226, United S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4bdcade72a3a0f470fecb3b6,MangoSeed Restaurant,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",v-1564173857,False,757 Flatbush Ave,40.654403,-73.959560,"[{'label': 'display', 'lat': 40.65440267666848...",969,11226,US,Brooklyn,NY,United States,"[757 Flatbush Ave (Between Lenox & Clarkson), ...",Between Lenox & Clarkson,108162,https://www.seamless.com/menu/mangoseed-757-fl...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,39496036,NaN
2,3fd66200f964a5207cf11ee3,Junior's Restaurant,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1564173857,False,386 Flatbush Avenue Ext,40.690011,-73.981734,"[{'label': 'display', 'lat': 40.69001145156075...",5206,11201,US,Brooklyn,NY,United States,"[386 Flatbush Avenue Ext (at DeKalb Ave), Broo...",at DeKalb Ave,291315,https://www.seamless.com/menu/juniors-restaura...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,66852513,Downtown Brooklyn
3,4bfafc771134b713e48c0fc9,Green Lake Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1564173857,False,795 Flatbush Ave,40.653110,-73.959343,"[{'label': 'display', 'lat': 40.65311049787105...",889,11226,US,Brooklyn,NY,United States,"[795 Flatbush Ave (btw Lenox & Caton), Brookly...",btw Lenox & Caton,23279,https://www.seamless.com/menu/green-lake-resta...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN
4,4a6c86e5f964a520b8d01fe3,Tom's Restaurant,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1564173857,False,782 Washington Ave,40.674537,-73.963216,"[{'label': 'display', 'lat': 40.67453698094471...",2953,11238,US,Brooklyn,NY,United States,"[782 Washington Ave (at Sterling Pl), Brooklyn...",at Sterling Pl,297390,https://www.seamless.com/menu/toms-restaurant-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,82422922,Prospect Heights


In [20]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_1 = ['name', 'categories'] + [col for col in dataframe_1.columns if col.startswith('location.')] + ['id']
dataframe_filtered_1 = dataframe_1.loc[:, filtered_columns_1]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_1['categories'] = dataframe_filtered_1.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_1.columns = [column_1.split('.')[-1] for column_1 in dataframe_filtered_1.columns]

#dataframe_filtered_1

In [21]:
dataframe_filtered_1.shape

(50, 16)

In [22]:
venues_map_1 = folium.Map(location=[latitude_1, longitude_1], zoom_start=11) 

# add a red circle marker to represent the Brooklyn
folium.features.CircleMarker(
    [latitude_1, longitude_1],
    radius=10,
    color='red',
    popup='Brooklyn',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_1)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_1.lat, dataframe_filtered_1.lng, dataframe_filtered_1.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_1)

# display map
venues_map_1

In [23]:
#categories count
dataframe_filtered_1.groupby('categories').count()

,name,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
categories,,,,,,,,,,,,,,,
African Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Asian Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Bakery,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
Bar,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Caribbean Restaurant,16,15,16,16,16,16,16,16,16,16,16,16,10,1,16
Chinese Restaurant,10,9,10,10,10,10,10,10,10,10,10,10,6,0,10
Diner,5,5,5,5,5,5,5,5,5,5,5,5,4,2,5
Food,2,2,2,2,2,2,2,2,2,2,2,2,0,0,2
Halal Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1


In [24]:
if len(results_1['response']['venues']) == 0:
    trending_venues_1_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues_1 = results_1['response']['venues']
    trending_venues_1_df = json_normalize(trending_venues_1)

    # filter columns
    columns_filtered_1 = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_1_df = trending_venues_1_df.loc[:, columns_filtered_1]

    # filter the category for each row
    trending_venues_1_df['categories'] = trending_venues_1_df.apply(get_category_type, axis=1)

In [25]:
if len(results_1['response']['venues']) == 0:
    trending_venues_map_1 = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    trending_venues_map_1 = folium.Map(location=[latitude_1, longitude_1], zoom_start=15) # generate map centred around Brooklyn


    # add Brooklyn as a red circle mark
    folium.features.CircleMarker(
        [latitude_1, longitude_1],
        radius=10,
        popup='Brooklyn',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(trending_venues_map_1)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_1_df['location.lat'], trending_venues_1_df['location.lng'], trending_venues_1_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(trending_venues_map_1)
        
        
 

In [26]:
# display map
trending_venues_map_1 

In [ ]:
#same process for Queens

In [27]:
queens_df = df_nygeo.loc[df_nygeo.Borough=="Queens"].reset_index(drop=True)
queens_df.shape

(81, 9)

In [28]:
address_2 = 'Queens, NY'

geolocator_2 = Nominatim(user_agent="qu_explorer")
location_2 = geolocator_2.geocode(address_2)
latitude_2 = location_2.latitude
longitude_2 = location_2.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude_2, longitude_2))

The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.


In [29]:
url_2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_2, longitude_2, VERSION, search_query, radius, LIMIT)
url_2

'https://api.foursquare.com/v2/venues/search?client_id=GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR&client_secret=MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3&ll=40.6524927,-73.7914214158161&v=20180604&query=restaurant&radius=5000&limit=300'

In [30]:
results_2 = requests.get(url_2).json()
#results_2

In [31]:
# assign relevant part of JSON to venues
venues_2 = results_2['response']['venues']

# tranform venues into a dataframe
dataframe_2 = json_normalize(venues_2)
#dataframe_2.head()

In [32]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_2 = ['name', 'categories'] + [col for col in dataframe_2.columns if col.startswith('location.')] + ['id']
dataframe_filtered_2 = dataframe_2.loc[:, filtered_columns_2]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_2['categories'] = dataframe_filtered_2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_2.columns = [column_2.split('.')[-1] for column_2 in dataframe_filtered_2.columns]

#dataframe_filtered_2

In [33]:
venues_map_2 = folium.Map(location=[latitude_2, longitude_2], zoom_start=11) # generate map centred around Queens

# add a red circle marker to represent Queens
folium.features.CircleMarker(
    [latitude_2, longitude_2],
    radius=10,
    color='red',
    popup='Queens',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_2)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_2.lat, dataframe_filtered_2.lng, dataframe_filtered_2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_2)

# display map
venues_map_2

In [45]:
dataframe_filtered_2.groupby('categories').count()

,name,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,postalCode,crossStreet,neighborhood,id
categories,,,,,,,,,,,,,,,
American Restaurant,3,3,3,3,3,3,3,3,3,3,1,3,0,0,3
Asian Restaurant,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1
Bakery,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Caribbean Restaurant,13,13,13,13,13,13,13,13,13,13,13,13,3,0,13
Chinese Restaurant,8,8,8,8,8,8,7,8,8,8,7,7,0,0,8
Food,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Greek Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1
Italian Restaurant,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2
Latin American Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1


In [34]:
if len(results_2['response']['venues']) == 0:
    trending_venues_2_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues_2 = results_2['response']['venues']
    trending_venues_2_df = json_normalize(trending_venues_2)

    # filter columns
    columns_filtered_2 = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_2_df = trending_venues_2_df.loc[:, columns_filtered_2]

    # filter the category for each row
    trending_venues_2_df['categories'] = trending_venues_2_df.apply(get_category_type, axis=1)

In [35]:
if len(results_2['response']['venues']) == 0:
    trending_venues_map_2 = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    trending_venues_map_2 = folium.Map(location=[latitude_2, longitude_2], zoom_start=11) # generate map centred around Queens


    # add Queens as a red circle mark
    folium.features.CircleMarker(
        [latitude_2, longitude_2],
        radius=10,
        popup='Queens',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(trending_venues_map_2)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_2_df['location.lat'], trending_venues_2_df['location.lng'], trending_venues_2_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(trending_venues_map_2)
        
        


In [36]:
# display map
trending_venues_map_2